In [10]:
# To-Do List Application for Google Colab
import json
import os
import uuid
from datetime import datetime

DATA_PATH = "/content/todo_tasks.json"

def load_tasks(path=DATA_PATH):
    if not os.path.exists(path):
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return []

def save_tasks(tasks, path=DATA_PATH):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(tasks, f, ensure_ascii=False, indent=2)

def create_task(title, description="", due_date=None):
    task = {
        "id": str(uuid.uuid4()),
        "title": title.strip(),
        "description": description.strip(),
        "created_at": datetime.utcnow().isoformat() + "Z",
        "due_date": due_date,
        "done": False,
        "done_at": None
    }
    return task

def list_tasks(tasks, show_all=True, show_done=True):
    filtered = tasks if show_all else [t for t in tasks if not t["done"]]
    if not show_done:
        filtered = [t for t in filtered if not t["done"]]
    if not filtered:
        print("No tasks to show.")
        return
    print("-" * 80)
    for i, t in enumerate(filtered, 1):
        print(f"[{i}] ID: {t['id']}")
        print(f"    Title      : {t['title']}")
        if t["description"]:
            print(f"    Description: {t['description']}")
        print(f"    Created    : {t['created_at']}")
        if t["due_date"]:
            print(f"    Due date   : {t['due_date']}")
        print(f"    Done       : {t['done']}")
        if t["done_at"]:
            print(f"    Done at    : {t['done_at']}")
        print("-" * 80)

def find_task_by_index(tasks, index, filtered=None):
    if filtered is None:
        filtered = tasks
    try:
        idx = int(index) - 1
        if idx < 0 or idx >= len(filtered):
            return None
        return filtered[idx]
    except Exception:
        return None

def find_task_by_id(tasks, task_id):
    for t in tasks:
        if t["id"] == task_id:
            return t
    return None

def parse_date(date_str):
    if not date_str:
        return None
    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%Y/%m/%d", "%d-%m-%Y"):
        try:
            d = datetime.strptime(date_str.strip(), fmt)
            return d.date().isoformat()
        except Exception:
            continue
    return None

def add_task_interactive(tasks):
    title = input("Title: ").strip()
    if not title:
        print("Title cannot be empty.")
        return
    description = input("Description (optional): ").strip()
    due = input("Due date (YYYY-MM-DD or DD/MM/YYYY) (optional): ").strip()
    due_parsed = parse_date(due)
    if due and not due_parsed:
        print("Could not parse due date. Save without due date? (y/n)")
        c = input().strip().lower()
        if c != "y":
            print("Add cancelled.")
            return
    task = create_task(title, description, due_parsed)
    tasks.append(task)
    save_tasks(tasks)
    print("Task added. ID:", task["id"])

def delete_task_interactive(tasks):
    if not tasks:
        print("No tasks to delete.")
        return
    print("Choose delete by: 1) index (list)  2) id")
    choice = input("Choice [1/2]: ").strip()
    if choice == "1":
        list_tasks(tasks)
        idx = input("Index to delete: ").strip()
        try:
            i = int(idx) - 1
            if i < 0 or i >= len(tasks):
                print("Index out of range.")
                return
            removed = tasks.pop(i)
            save_tasks(tasks)
            print("Deleted task:", removed["title"])
        except Exception:
            print("Invalid index.")
    elif choice == "2":
        tid = input("Task ID to delete: ").strip()
        t = find_task_by_id(tasks, tid)
        if not t:
            print("Task ID not found.")
            return
        tasks.remove(t)
        save_tasks(tasks)
        print("Deleted task:", t["title"])
    else:
        print("Invalid choice.")

def mark_done_interactive(tasks):
    if not tasks:
        print("No tasks available.")
        return
    list_tasks(tasks, show_all=True)
    tid = input("Enter task ID to mark done (or index number): ").strip()
    target = find_task_by_id(tasks, tid)
    if not target:
        target = find_task_by_index(tasks, tid, filtered=tasks)
    if not target:
        print("Task not found.")
        return
    if target["done"]:
        print("Task is already marked done.")
        return
    target["done"] = True
    target["done_at"] = datetime.utcnow().isoformat() + "Z"
    save_tasks(tasks)
    print("Task marked as done:", target["title"])

def unmark_done_interactive(tasks):
    if not tasks:
        print("No tasks available.")
        return
    done_tasks = [t for t in tasks if t["done"]]
    if not done_tasks:
        print("No tasks marked as done.")
        return
    list_tasks(done_tasks)
    tid = input("Enter task ID to unmark done (or index number): ").strip()
    target = find_task_by_id(tasks, tid)
    if not target:
        target = find_task_by_index(tasks, tid, filtered=done_tasks)
        if target:
            target = find_task_by_id(tasks, target["id"])
    if not target:
        print("Task not found.")
        return
    target["done"] = False
    target["done_at"] = None
    save_tasks(tasks)
    print("Task unmarked as done:", target["title"])

def edit_task_interactive(tasks):
    if not tasks:
        print("No tasks available.")
        return
    list_tasks(tasks)
    tid = input("Enter task ID to edit (or index number): ").strip()
    target = find_task_by_id(tasks, tid)
    if not target:
        target = find_task_by_index(tasks, tid, filtered=tasks)
    if not target:
        print("Task not found.")
        return
    print("Leave field empty to keep current value.")
    new_title = input(f"Title [{target['title']}]: ").strip()
    new_description = input(f"Description [{target.get('description','')}]: ").strip()
    new_due = input(f"Due date [{target.get('due_date', '')}]: ").strip()
    if new_title:
        target["title"] = new_title
    if new_description:
        target["description"] = new_description
    if new_due:
        parsed = parse_date(new_due)
        if parsed:
            target["due_date"] = parsed
        else:
            print("Could not parse new due date. Keeping old value.")
    save_tasks(tasks)
    print("Task updated:", target["title"])

def import_from_csv_interactive(tasks):
    print("Provide the full path to CSV file (columns: title,description,due_date [optional])")
    path = input("CSV path: ").strip()
    if not os.path.exists(path):
        print("File not found.")
        return
    try:
        import csv
        added = 0
        with open(path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                title = row.get("title") or row.get("Title")
                if not title:
                    continue
                description = row.get("description", "")
                due = parse_date(row.get("due_date", "") or row.get("Due_date", ""))
                tasks.append(create_task(title, description, due))
                added += 1
        save_tasks(tasks)
        print(f"Imported {added} tasks.")
    except Exception as e:
        print("Import failed:", e)

def export_to_csv_interactive(tasks):
    print("Provide the path to save CSV file.")
    path = input("Save CSV to: ").strip()
    try:
        import csv
        with open(path, "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["id", "title", "description", "created_at", "due_date", "done", "done_at"])
            for t in tasks:
                writer.writerow([t["id"], t["title"], t["description"], t["created_at"], t["due_date"], t["done"], t["done_at"]])
        print("Exported tasks to", path)
    except Exception as e:
        print("Export failed:", e)

def show_menu():
    print("\nTo-Do List - Menu")
    print("1. List all tasks")
    print("2. List pending tasks")
    print("3. Add a task")
    print("4. Edit a task")
    print("5. Delete a task")
    print("6. Mark task as done")
    print("7. Unmark done")
    print("8. Import from CSV")
    print("9. Export to CSV")
    print("0. Exit\n")

def main():
    tasks = load_tasks()
    while True:
        try:
            show_menu()
            choice = input("Select option: ").strip()
            if choice == "1":
                list_tasks(tasks, show_all=True)
            elif choice == "2":
                list_tasks(tasks, show_all=False)
            elif choice == "3":
                add_task_interactive(tasks)
            elif choice == "4":
                edit_task_interactive(tasks)
            elif choice == "5":
                delete_task_interactive(tasks)
            elif choice == "6":
                mark_done_interactive(tasks)
            elif choice == "7":
                unmark_done_interactive(tasks)
            elif choice == "8":
                import_from_csv_interactive(tasks)
            elif choice == "9":
                export_to_csv_interactive(tasks)
            elif choice == "0":
                print("Exiting. Goodbye.")
                break
            else:
                print("Invalid option.")
        except KeyboardInterrupt:
            print("\nInterrupted. Exiting.")
            break
        except Exception as e:
            print("An error occurred:", e)

if __name__ == "__main__":
    main()



To-Do List - Menu
1. List all tasks
2. List pending tasks
3. Add a task
4. Edit a task
5. Delete a task
6. Mark task as done
7. Unmark done
8. Import from CSV
9. Export to CSV
0. Exit

Select option: 4
--------------------------------------------------------------------------------
[1] ID: 62763196
    Title      : project about to do
    Description: fffffffffffffffffffffffffffffffffffffffffffffffffffffffff
    Created    : 2025-08-28T17:45:53.380059
    Due date   : 33/000/988
    Done       : False
--------------------------------------------------------------------------------
[2] ID: 7c1302c7
    Title      : Htaa  task
    Description: I have 1 year of experience in web development with strong skills in *JavaScript, CSS, and HTML*. I’m passionate about creating clean, modern, and fully responsive websites that deliver a seamless user experience.
    Created    : 2025-08-28T18:08:09.681971
    Due date   : 28/8/2055
    Done       : False
-----------------------------------------

In [9]:
!pip install gradio pandas

import gradio as gr
import json, os, uuid, pandas as pd
from datetime import datetime

DATA_PATH = "todo_tasks.json"

def load_tasks():
    if not os.path.exists(DATA_PATH):
        return []
    try:
        with open(DATA_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    except:
        return []

def save_tasks(tasks):
    with open(DATA_PATH, "w", encoding="utf-8") as f:
        json.dump(tasks, f, ensure_ascii=False, indent=2)

def create_task(title, description="", due_date=""):
    tasks = load_tasks()
    task = {
        "id": str(uuid.uuid4())[:8],
        "title": title.strip(),
        "description": description.strip(),
        "due_date": due_date.strip(),
        "created_at": datetime.utcnow().isoformat(),
        "done": False,
        "done_at": None
    }
    tasks.append(task)
    save_tasks(tasks)
    return list_tasks()

def list_tasks():
    tasks = load_tasks()
    if not tasks:
        return "No tasks yet."
    out = ""
    for t in tasks:
        status = "✔️ Done" if t["done"] else "⏳ Pending"
        out += f"ID: {t['id']}\nTitle: {t['title']}\nDesc: {t['description']}\nDue: {t['due_date']}\nStatus: {status}\n---\n"
    return out

def mark_done(task_id):
    tasks = load_tasks()
    for t in tasks:
        if t["id"] == task_id:
            t["done"] = True
            t["done_at"] = datetime.utcnow().isoformat()
            save_tasks(tasks)
            return list_tasks()
    return "Task not found"

def unmark_done(task_id):
    tasks = load_tasks()
    for t in tasks:
        if t["id"] == task_id:
            t["done"] = False
            t["done_at"] = None
            save_tasks(tasks)
            return list_tasks()
    return "Task not found"

def delete_task(task_id):
    tasks = load_tasks()
    new_tasks = [t for t in tasks if t["id"] != task_id]
    if len(new_tasks) == len(tasks):
        return "Task not found"
    save_tasks(new_tasks)
    return list_tasks()

def edit_task(task_id, new_title="", new_desc="", new_due=""):
    tasks = load_tasks()
    for t in tasks:
        if t["id"] == task_id:
            if new_title.strip():
                t["title"] = new_title.strip()
            if new_desc.strip():
                t["description"] = new_desc.strip()
            if new_due.strip():
                t["due_date"] = new_due.strip()
            save_tasks(tasks)
            return list_tasks()
    return "Task not found"

def export_csv():
    tasks = load_tasks()
    if not tasks:
        return None
    df = pd.DataFrame(tasks)
    path = "tasks.csv"
    df.to_csv(path, index=False)
    return path

def import_csv(file):
    if file is None:
        return "No file uploaded"
    df = pd.read_csv(file.name)
    tasks = load_tasks()
    for _, row in df.iterrows():
        task = {
            "id": str(uuid.uuid4())[:8],
            "title": str(row.get("title", "")),
            "description": str(row.get("description", "")),
            "due_date": str(row.get("due_date", "")),
            "created_at": datetime.utcnow().isoformat(),
            "done": False,
            "done_at": None
        }
        tasks.append(task)
    save_tasks(tasks)
    return list_tasks()

with gr.Blocks() as demo:
    gr.Markdown("## 📝 To-Do List Web App (Complete Version)")

    with gr.Tab("Add Task"):
        title = gr.Textbox(label="Task Title")
        desc = gr.Textbox(label="Description")
        due = gr.Textbox(label="Due Date (YYYY-MM-DD)")
        add_btn = gr.Button("Add Task")
        out_add = gr.Textbox(label="Tasks", lines=15)
        add_btn.click(create_task, inputs=[title, desc, due], outputs=out_add)

    with gr.Tab("Show Tasks"):
        show_btn = gr.Button("Refresh Task List")
        out_list = gr.Textbox(label="All Tasks", lines=20)
        show_btn.click(list_tasks, outputs=out_list)

    with gr.Tab("Edit Task"):
        task_id = gr.Textbox(label="Task ID to Edit")
        new_title = gr.Textbox(label="New Title")
        new_desc = gr.Textbox(label="New Description")
        new_due = gr.Textbox(label="New Due Date")
        edit_btn = gr.Button("Save Changes")
        out_edit = gr.Textbox(label="Tasks", lines=15)
        edit_btn.click(edit_task, inputs=[task_id, new_title, new_desc, new_due], outputs=out_edit)

    with gr.Tab("Actions"):
        tid = gr.Textbox(label="Task ID")
        done_btn = gr.Button("Mark as Done")
        undone_btn = gr.Button("Unmark Done")
        del_btn = gr.Button("Delete Task")
        out_act = gr.Textbox(label="Tasks", lines=15)
        done_btn.click(mark_done, inputs=tid, outputs=out_act)
        undone_btn.click(unmark_done, inputs=tid, outputs=out_act)
        del_btn.click(delete_task, inputs=tid, outputs=out_act)

    with gr.Tab("Import/Export"):
        upload = gr.File(label="Import CSV", file_types=[".csv"])
        import_btn = gr.Button("Import")
        out_imp = gr.Textbox(label="Tasks", lines=15)
        import_btn.click(import_csv, inputs=upload, outputs=out_imp)

        export_btn = gr.Button("Export CSV")
        file_out = gr.File()
        export_btn.click(export_csv, outputs=file_out)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://450d00b825c4c4aae3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
